# **01. Preparing data needed for calc_hdx and HDXer**

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Matplotlib settings for plotting
plt.rc('lines', linewidth=3, markersize=4)
plt.rc('axes', labelweight='heavy', labelsize=22, titlesize=22) # Default fontsizes for printing
plt.rc('axes.spines', top=False, right=False) # Switch off top/right axes
plt.rc('legend', fontsize=16) # Default fontsizes for printing
plt.rc('xtick', labelsize=16) # Default fontsizes for printing
plt.rc('ytick', labelsize=16) # Default fontsizes for printing
plt.rc('figure', titlesize=22, titleweight='heavy') # Default fontsizes for printing

Hydrogen-deuterium exchange (HDX) is a popular biophysical tool used to study the structure and dynamics of proteins in their native or near-native environments. Typically, HDX experiments are designed to measure the exchange of protein backbone amide hydrogen atoms for deuterium atoms from a bulk deuterated solution. The rate of H-D exchange is governed by chemical factors, such as steric and inductive effects of the neighboring sidechains upon the amide functional group, and by physical factors, such as structural and dynamical protection afforded by the equilibrium motions of the protein in solution. Measuring, comparing, and rationalizing rates of exchange can therefore provide insights into the structures and dynamics of proteins under a desired set of environmental conditions. Deuterium labeling from H-D exchange can be measured directly by biophysical tools such as NMR, but most recently has been popularized with mass spectrometry (HDX-MS), which avoids many limitations of protein size, concentration, and throughput, associated with other experimental techniques.

***HDXer*** is a Python package designed to 1) Compute HDX exchange rates and observed HDX-MS measurements from biomolecular simulation data using empirical predictive models, and 2) Aid the structural interpretation of HDX-MS experiments _via_ a Maximum-Entropy-based reweighting of simulated ensembles to best fit experimental data. These two steps are divided into separate modules of the ***HDXer*** package, which can be imported individually so that users can design and apply customized ***HDXer*** analyses. Alternatively, the ***HDXer*** functionality of step (1) can be accessed using a separate Python wrapper script: **calc_hdx.py**. In this suite of tutorials, we will calculate HDX deuterated fractions of a given structural ensemble using **calc_hdx.py**, and then reweight an ensemble of candidate structures to match up with experimental HDX-MS data using the ***HDXer*** modules in an interactive Python script.

Given the vast range of software and approaches available to perform MD simulations and HDX experiments, the formats of structural ensembles and experimental HDX data may need to be pre-processed and standardized prior to using them with ***HDXer***. This notebook provides instructions on which data are needed, and how to process them into the formats required by ***HDXer***.

There are four necessary data files to run ***HDXer***:
- An atomistic ensemble of protein structures (e.g. from a molecular dynamics trajectory)
- A protein topology (e.g. a PDB file)
- A target, usually experimental, set of HDX-MS data (measured as HDX deuterated fractions for defined peptide segments at specific deuterium labeling times)
- A list of peptide segments present in the experimental HDX-MS data

We will use published data on BPTI (Bovine Pancreatic Trpysin Inhibitor) as an example system. BPTI is one of the smallest and simplest globular proteins, which provides us with two advantages:

1. Typically, bottom-up HDX-MS data is collected for protein segments that are multiple residues in length. However, for BPTI, there is NMR-based HDX data available with single-residue resolution and a large coverage of the protein.

[Persson, F. and Halle, B. (2015) ‘How amide hydrogens exchange in native proteins’, Proceedings of the National Academy of Sciences, 112(33), 10383-10388. doi: 10.1073/pnas.1506079112.](https://www.pnas.org/content/112/33/10383.long)

[Kim, K.S., Fuchs, J.A., and Woodward, C.K. (1993) 'Hydrogen exchange identifies native-state motional domains important in protein folding', Biochemistry, 32(37), 9600-9608](https://pubs.acs.org/doi/abs/10.1021/bi00088a012)

[Battiste, J.L., Li, R., and Woodward, C. (2002) 'A Highly Destabilizing Mutation, G37A, of the Bovine Pancreatic Trypsin Inhibitor Retains the Average Native Conformation but Greatly Increases Local Flexibility', Biochemistry, 41(7), 2237-2245](https://pubs.acs.org/doi/full/10.1021/bi011693e?mobileUi=0)

2. As a small protein with numerous atomic structures solved by either X-ray crystallography or NMR, BPTI is highly amenable to study with lengthy unbiased MD simulations. In a later notebook we will provide example trajectories of BPTI that can be used to compute HDX-MS values for comparison to experiment.

The experimental HDX data for BPTI were first reported (in the articles above by Woodward and coworkers) as observed H-D exchange rate constants for each amide, measured by NMR. However, rate constants for individual residues were variously measured at different temperatures and pD values. Persson & Halle converted the observed rate constants to equivalent temperature and pD measurements, and then reported the same data as an experimental 'protection factor' for each residue. The protection factor (PF, or $P_i$) is a unitless constant that describes the extent to which a residue's _intrinsic_ rate of exchange is modulated by the structure and dynamics of its environment. **HDXer** requires input experimental data formatted as peptide-segment deuterated fractions at given points in time, as this is the more frequent experimental observable from HDX-MS experiments. We must therefore convert these experimental protection factors to HDX deuterated fractions at given points in time. Let's first open and look at the original data.

In [ ]:
# Load experimental protection factors
expt_data = pd.read_csv(os.path.expandvars('$HDXER_PATH/tutorials/BPTI/BPTI_expt_data/BPTI_expt_PFs.dat'), 
                        sep='\s+', skiprows=[0], header=None, names=['Residue', 'log10(PF)'])

We can see how the experimental data is structured by looking at the first five rows. The first column is the residue number and the second column is the corresponding logarithm (in base 10) of the protection factor.

In [ ]:
expt_data.head()

Using the equation below, we can convert these experimental constants to HDX deuterated fractions.

\begin{equation*}
D_{i,t} = 1 - exp(\frac{-k_{int,i}}{P_{i}}t)
\end{equation*}

Therefore, to convert the measured protection factors we still need two values, the intrinsic rate constants of exchange ($k_{int}$), and the time points at which to calculate deuteration (t).

Normally, these time points would arise from the labeling times used in an HDX-MS experiment. However, because, in this case, we are converting from protection factors obtained by NMR, we can choose any timepoints at all. For simplicity, we will use time points that are the default used by **calc_hdx.py** for HDX-MS calculations, if no other labeling times are specified. These are 0.167, 1.0, 10.0, and 120.0 min:

In [ ]:
times = np.array([0.167, 1.0, 10.0, 120.0]) # Create a NumPy array with time points in min                                                                                                                                                                                                                           utes

As the protection factors define the structural and dynamical protection afforded to each residue by the folded protein environment, the intrinsic rate constants of exchange represent the rate at which each residue would exchange in an unfolded, unprotected, system. Intrinsic amide exchange rate constants therefore depend upon the steric and inductive effects arising from neighboring sidechains, and the temperature and pD of the reaction solution. In this case, we provide a list of the by-residue intrinsic exchange rate constants for BPTI at pD 7.4 and 298 K, available as "BPTI_Intrinsic_rates.dat". However, for future reference, **calc_hdx.py** can also be used to calculate the intrinsic exchange rate constants directly from provided protein topology and coordinate files. The details of how to run **calc_hdx.py** will be discussed in the next notebook.

For now, let's read in the intrinsic rates file and use it to calculate deuterated fractions:

In [ ]:
# Load intrinsic rates
rates = pd.read_csv(os.path.expandvars('$HDXER_PATH/tutorials/BPTI/BPTI_expt_data/BPTI_Intrinsic_rates.dat'), 
                    sep='\s+', skiprows=[0], header=None, names=['Residue', 'k_int'])

In [ ]:
rates.head()

Deuterated fractions can only be calculated for residues for which both protection factors and intrinsic rates are available. Therefore we have to apply a filter to the intrinsic rates dataset:

In [ ]:
expt_data['k_int'] = rates[rates['Residue'].isin(expt_data['Residue'])]['k_int'].values

In [ ]:
expt_data.head()

Now, we can calculate deuterated fractions for each residue and each time point using the simple function below:

In [ ]:
def residue_fraction(pf, k, t):
    """Calculate deuterated fractions at time t using protection factors and intrinsic rates. 
    
    Args:
        pf: log10(PF)
        k: intrinsic rate (min^-1)
        t: time in minutes
        
    Returns:
        val: HDX deuterated fractions for each residue and time point
    
    Usage: residue_fraction(rate, t)
    """
    logf = (-k) / (10**pf) * t
    val = 1 - np.exp(logf)
    return val

In [ ]:
dfracs = np.zeros((len(times), len(expt_data))) # Create a NumPy array with zeros which we will add deuterated fractions to

for i, t in enumerate(times):
    for j, (pf, k) in enumerate(zip(expt_data['log10(PF)'], expt_data['k_int'])):
        dfracs[i, j] = residue_fraction(pf, k, t)

Note that this function, and the first-order rate law that it implements, assumes that each residue can eventually reach full deuteration (i.e. the maximal deuterated fraction is 1.0). Experimentally, this would be the case if, for example, labeling took place in 100% D$_2$O. However, in many experiments, mixing of buffer volumes means that this is not the case, and instead the D$_2$O content of the labeling buffer is lower, perhaps 80-90%. Any experimental HDX-MS data used with HDXer should always be normalized, as if it had been measured in 100% D$_2$O, and corrected for back-exchange, prior to the HDXer analysis - we recommend following the [community guidelines](https://doi.org/10.1038/s41592-019-0459-y) for a great explanation of how to correct and normalize experimental HDX-MS data.

For the data used in the tutorials however, no normalization or correction is needed. The next step is then to save the calculated deuterated fractions in a text file, ready for visualization later on, and use in the subsequent tutorials. We'll save the file in the format required by HDXer for ensemble reweighting predictions in the next tutorial. Data for each residue is stored The first two columns in the file represent the residue number of the first and last residue in each peptide segment (remember - we're formatting our HDXer input data as if they had been generated by HDX-MS!), and the next four columns represent the deuterated fraction of that peptide segment at each labeling time, $t$:

In [ ]:
# Save HDX deuterated fractions, with 2 fields for residue start/end
np.savetxt(os.path.expandvars('$HDXER_PATH/tutorials/BPTI/BPTI_expt_data/BPTI_expt_dfracs.dat'), \
           np.concatenate((np.atleast_2d(expt_data['Residue'] - 1), np.atleast_2d(expt_data['Residue']), dfracs)).T, \
           fmt="%3d %3d %8.5f %8.5f %8.5f %8.5f", \
           header="ResStr, ResEnd, Deuterated fractions, Times / min: %s" \
           % ' '.join([ str(t) for t in times ]))

Note that to convert our residues to 'peptide segments', we create segments that are two residues in length, from residue $i-1$ to residue $i$. This is to make sure we are consistent with the context of HDX-MS, in which we expect the N-terminal residue of each peptide segment to be completely back-exchanged to hydrogen during the quenching, digestion, and analysis process. A segment from residue $i-1$ to residue $i$ therefore only reports upon the deuteration of residue $i$

So, now we have a new file called *BPTI_expt_dfracs.dat* that contains experimental HDX data. Let's plot this to visualize what experimental HDX data looks like:

There are two Python functions we'll use throughout the tutorial to plot and visualize HDX deuterated fractions. We'll repeat them each time, so feel free to adapt the functions to suit your own plotting preferences, or to read in different files and investigate the data in more detail.

The first function reads in HDXer datafiles, such as the list of peptide segments or HDX deuterated fractions, and returns the contents as a pandas dataframe:

In [ ]:
def read_to_df(file):
    """Read and create a pandas DataFrame for the given argument.
    
    Args:
        file: There are four possible options:
            'segs' - peptide segments
            'expt' - experimental HDX deuterated fractions
            'pred' - calculated HDX deuterated fractions
            'reweighted' - reweighted HDX deuterated fractions
    
    Returns:
        df: A pandas DataFrame containing data for the given argument.
    """
    if file == 'segs':
        # Read and create a pandas DataFrame using a residue segments file
        df = pd.read_csv(os.path.expandvars('$HDXER_PATH/tutorials/BPTI/BPTI_expt_data/BPTI_residue_segs.txt'),
                         sep='\s+', header=None, names=['ResStr', 'ResEnd'])
    elif file == 'expt':
        # Read and create a pandas DataFrame using an experimental deuterated fractions file
        df = pd.read_csv(os.path.expandvars('$HDXER_PATH/tutorials/BPTI/BPTI_expt_data/BPTI_expt_dfracs.dat'), 
                         sep='\s+', skiprows=[0], header=None, usecols=[2, 3, 4, 5], names=times)
    elif file == 'pred':
        # Read and create a pandas DataFrame using a computed deuterated fractions file
        df = pd.read_csv(os.path.expandvars('$HDXER_PATH/tutorials/BPTI/BPTI_calc_hdx/BPTI_SUMMARY_segment_average_fractions.dat'), 
                         sep='\s+', skiprows=[0], header=None, usecols=[2, 3, 4, 5], names=times)
    elif file == 'reweighted':
        df = pd.read_csv(os.path.expandvars('$HDXER_PATH/tutorials/BPTI/BPTI_reweighting/reweighting_gamma_2x10^0_final_segment_fractions.dat'), 
                         sep='\s+', skiprows=[0], header=None, names=times)
    else:
        print("Incorrect argument given. Please choose one of the following: 'segs' 'expt' 'pred' 'reweighted'")
    return df

### **Plotting protection factors and intrinsic rates**

As an example, we can use this function to read in the protection factor and intrinsic rate datafiles in a a dataframe, and visualize the contents.

In [ ]:
fig, ax = plt.subplots(figsize=(11, 8.5))
xs = np.arange(0, read_to_df('segs').iloc[:, 1].shape[0])
ax.plot(xs, expt_data['log10(PF)'])
ax.set_title('BPTI log protection factor per residue')
ax.set_xlabel('Residue')
ax.set_ylabel('log$_{10}$PF')
ax.set_xticks(xs)
ax.set_xlim(xs[0], xs[-1])
ax.set_xticklabels(read_to_df('segs').iloc[:, 1], rotation=90)
plt.savefig('BPTI_expt_PFs.pdf', bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=(11, 8.5))
xs = np.arange(0, read_to_df('segs').iloc[:, 1].shape[0])
ax.plot(xs, expt_data['k_int'])
ax.set_title('BPTI intrinsic rate per residue')
ax.set_xlabel('Residue')
ax.set_ylabel(r'Intrinsic rate constant / min$^{-1}$')
ax.set_xticks(xs)
ax.set_xlim(xs[0], xs[-1])
ax.set_xticklabels(read_to_df('segs').iloc[:, 1], rotation=90);
plt.savefig('BPTI_k_int.pdf', bbox_inches='tight')

The second function plots HDX deuterated fractions for each time point.

In [ ]:
def plot_dfracs(*args):
    """Plot HDX deuterated fractions for each time point.
    
    Args:
        *args: 
            'expt' - experimental HDX deuterated fractions
            'pred' - computed HDX deuterated fractions
            'reweighted' - reweighted HDX deuterated fractions
    """
    fig, axs = plt.subplots(len(times), 1, figsize=(12, 24))
    for i, (ax, t) in enumerate(zip(axs, times)):
        for arg in args:
            if arg in ['expt', 'pred', 'reweighted']:
                xs = np.arange(0, read_to_df(arg).iloc[:, 1].shape[0])
                ax.plot(xs, read_to_df(arg).iloc[:, i], label=arg)
                ax.set_title(f'Labeling time = {t} min')
                ax.legend(loc='upper right')
                ax.set_xticks(xs)
                ax.set_xlim(xs[0], xs[-1])
                ax.set_xticklabels(read_to_df('segs').iloc[:, 1], rotation=90)
                ax.set_ylim(0, 1)
            else:
                print("Incorrect argument given. Please choose one or more of the following: 'expt' 'pred' 'reweighted'")
    fig.text(0.5, 0.095, 'Residue', ha='center', fontsize=22)
    fig.text(0.05, 0.5, 'HDX deuterated fractions', va='center', rotation='vertical', fontsize=22)

In [ ]:
plot_dfracs('expt')

As you can see, residues in BPTI exhibit a wide range of protection factors, which results in a wide range of deuterated fraction values under experimental conditions of pD 7.4 and 298 K. Some residues exchange almost immediately, after 10 s labeling time, while many others remain protected even after 2 hours of D$_2$O labeling.

So far, we have organized our data directories and converted NMR-based experimental HDX protection factor data to HDX-MS-style deuterated fractions for each residue in BPTI. In the next notebook *02_calc_hdx.ipynb*, we will estimate HDX-MS deuterated fractions from a series of BPTI MD trajectories.